# tests format - pointer

In [1]:
from json_ntv import Ntv
from json_ntv.ntv_util import NtvUtil
from jsonpointer import resolve_pointer

examples = [ {'data': {'a': 1, 'test': 'ok'}, 'pointer': '/test', 'canonical': '/1'},
             {'data': [{'a': 1}, 'ok'],       'pointer': '/1',    'canonical': '/1'},
             {'data': {'a': 1, '1': 'ok'},    'pointer': '/1',    'canonical': '/1'},
             {'data': ['a', 'ok'],            'pointer': '/1',    'canonical': '/1'}]
equal = True

for example in examples:
    data      = example['data']
    pointer   = example['pointer']
    canonical = example['canonical']

    ntv = Ntv.obj(data)
    equal &= ntv['#'+canonical] == ntv['#'+pointer]

    equal &= ntv['#'+pointer].val == resolve_pointer(data, pointer) == 'ok'
equal

True

In [2]:
examples = [ {'data': {'test': 'ok'},                    'pointer': 'test' ,     'pointer json': '/test'},
             {'data': [1, {'test': 'ok'}],               'pointer': '/test',     'pointer json': '/1/test'},
             {'data': [1, [2, {'test': 'ok'}]],          'pointer': '/1/test',   'pointer json': '/1/1/test'},
             {'data': [1, {'a': {'test': 'ok'}}],        'pointer': '/a/test',   'pointer json': '/1/a/test'},
             {'data': {'b': [1, {'a': {'test': 'ok'}}]}, 'pointer': 'b/a/test',  'pointer json': '/b/1/a/test'},
             {'data': {'b': [1, {'a': {'test': 'ok'}}]}, 'pointer': '0/a/test',  'pointer json': '/b/1/a/test'},
             {'data': {'b': {'a': 1, 'test': 'ok'}},     'pointer': 'b/test',    'pointer json': '/b/test'},
             {'data': {'a': 1, 'test': 'ok'},            'pointer': '/test',     'pointer json': '/test'},
             {'data': [1, {'a': {'test': [1,'ok']}}],    'pointer': '/a/test/1', 'pointer json': '/1/a/test/1'}]
equal = True

for example in examples:
    data         = example['data']
    pointer      = example['pointer']
    pointer_json = example['pointer json']
    unique_root  = len(data) == 1

    ntv = Ntv.obj(data)
    equal &= ntv['#'+pointer].val == resolve_pointer(data, pointer_json)
    if pointer[0] != '0':
        equal &= NtvUtil.to_ntvpointer(pointer_json, unique_root) == pointer
equal

True

In [3]:
examples = [ 
             {'data': {'b': [1, {'a': {'test': 'ok'}}]}, 'pointer': 'b/a/test',  'pointer json': '/b/1/a/test'} ]
equal = True

for example in examples:
    data         = example['data']
    pointer      = example['pointer']
    pointer_json = example['pointer json']
    unique_root  = len(data) == 1
    
    ntv = Ntv.obj(data)
    print(ntv['#0'])
    equal &= ntv['#'+pointer].val == resolve_pointer(data, pointer_json)
    if pointer[0] != '0':
        equal &= NtvUtil.to_ntvpointer(pointer_json, unique_root) == pointer
equal

{"b": [1, {"a": {"test": "ok"}}]}


True

In [4]:
import json_ntv
from json_ntv import Namespace, Datatype
from namespace import from_file
import pathlib

file = pathlib.Path(json_ntv.__file__).parent.parent / "RFC" / "NTV_openAPI_namespace.ini"
from_file(file, '$openAPI.')

data = {"$openAPI.": {'servers.': {":url": "adr1", "test": "ok" }}}
ntv = Ntv.obj(data)

print(ntv[0][1].pointer(), ntv[0][1].pointer(index=True))
ntv['#$openAPI./servers./test'].val == resolve_pointer(data, '/$openAPI./servers./test')

$openAPI./servers./test 0/0/1


True

In [5]:
import re
p = re.compile('/([0-9]+)(/[a-z])')
reg = '/b/1/a/test'
reg = '21/b/123/abcd/test/456/45/rte'
#reg = '/b/c123/abcd/test'
print(p.findall(reg))
print(re.sub(r'/([0-9]+)(/[a-z])', r'\g<2>', reg))
print(re.sub('/([0-9]+)(/[a-z])', '\g<2>', reg))
p.sub('\g<2>',reg)

[('123', '/a'), ('45', '/r')]
21/b/abcd/test/456/rte
21/b/abcd/test/456/rte


'21/b/abcd/test/456/rte'

In [6]:
def single(match):
    return '/'
p.sub(single, reg)

'21/b/bcd/test/456/te'

In [7]:
q = re.compile('^(//)')
reg = '//b/1/a/test'
#reg = '/21/b/123/abcd/test/456/45/rte'

q.findall(reg)
q.sub('/', reg)
print(re.sub(r'^(//)', '/', reg))
print(re.sub('^(//)', '/', reg))

/b/1/a/test
/b/1/a/test
